<a href="https://colab.research.google.com/github/brunofbpaula/DataScience-UM-Coursera/blob/main/Pandas/DataFrame/PivotTable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pivot Table

It's a way of summarizing data in a DataFrame for particular purpose. It makes heavy use of the aggregation fuction. A pivot table is itself a DataFrame, where the rows represent one variable you're interested in, the columns another, and the cell's some aggregate value. A pivot table also tends to include marginal values as well, which are the sums of each column and row. This allows us to be able to see the relationship between two variables at just a glance.

In [1]:
import pandas as pd
import numpy as np

### Times Higher Education World University Ranking

A dataset that's the most influential university measures.

In [2]:
wur = pd.read_csv('cwurData.csv')
wur.head()

,world_rank,institution,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year
0,1,Harvard University,USA,1,7,9,1,1,1,1,NaN,5,100.00,2012
1,2,Massachusetts Institute of Technology,USA,2,9,17,3,12,4,4,NaN,1,91.67,2012
2,3,Stanford University,USA,3,17,11,5,4,2,2,NaN,15,89.50,2012
3,4,University of Cambridge,United Kingdom,1,10,24,4,16,16,11,NaN,50,86.17,2012
4,5,California Institute of Technology,USA,4,2,29,7,37,22,22,NaN,18,85.21,2012


We can see each institution's rank, country, quality of education, other metrics and overall score.

In [19]:
# Creating a new column called rank_level

# Creating a function that return the university's tier based on
# its position in world ranking
def category(ranking):
  if ranking <= 100:
    return 'First Tier'
  elif ranking <= 200:
    return 'Second Tier'
  elif ranking <= 300:
    return 'Third Tier'
  else:
    return 'Top Tier'

# Applying it to a single column and creating a new Series
wur['rank_level'] = wur['world_rank'].apply(lambda y: category(y))
wur.loc[299]

world_rank                                        100
institution             Royal Institute of Technology
country                                        Sweden
national_rank                                       2
quality_of_education                              185
alumni_employment                                  26
quality_of_faculty                                106
publications                                      223
influence                                         312
citations                                         310
broad_impact                                    326.0
patents                                           637
score                                           51.34
year                                             2014
rank_level                                 First Tier
Name: 299, dtype: object

A pivot table makes it possible to pivot out one of these columns a new column headers and compare it against another column as row indices.

In [21]:
# Let's say we want to compare rank level versus country of the universities and
# we want to compare in terms of overall score. Then we specify that the agg function,
# and we'll use numpy mean to get the average rating for universities in that country

wur.pivot_table(values='score', index='country', columns='rank_level', aggfunc=[np.mean]).head()

mean                                  
rank_level First Tier Second Tier Third Tier   Top Tier
country                                                
Argentina         NaN         NaN        NaN  44.672857
Australia     47.9425     49.2425  47.285000  44.645750
Austria           NaN         NaN  47.066667  44.864286
Belgium       51.8750     49.0840  46.746667  45.081000
Brazil            NaN     49.5650        NaN  44.499706

NaN values here indicate that the country doesn't have any observation in that category.

In [23]:
# Adding the max values
wur.pivot_table(values='score', index='country', columns='rank_level', aggfunc=[np.mean, np.max]).head()

mean                                         amax  \
rank_level First Tier Second Tier Third Tier   Top Tier First Tier   
country                                                              
Argentina         NaN         NaN        NaN  44.672857        NaN   
Australia     47.9425     49.2425  47.285000  44.645750      51.61   
Austria           NaN         NaN  47.066667  44.864286        NaN   
Belgium       51.8750     49.0840  46.746667  45.081000      52.03   
Brazil            NaN     49.5650        NaN  44.499706        NaN   

                                            
rank_level Second Tier Third Tier Top Tier  
country                                     
Argentina          NaN        NaN    45.66  
Australia        50.40      47.47    45.97  
Austria            NaN      47.78    46.29  
Belgium          49.73      47.14    46.21  
Brazil           49.82        NaN    46.08

In [25]:
# Summarizing the values within a given top level column
# Overall average for the country for the mean and max of the max

wur.pivot_table(values='score', index='country', columns='rank_level', aggfunc=[np.mean, np.max],
                margins=True).head()

mean                                                    amax  \
rank_level First Tier Second Tier Third Tier   Top Tier        All First Tier   
country                                                                         
Argentina         NaN         NaN        NaN  44.672857  44.672857        NaN   
Australia     47.9425     49.2425  47.285000  44.645750  45.825517      51.61   
Austria           NaN         NaN  47.066667  44.864286  45.139583        NaN   
Belgium       51.8750     49.0840  46.746667  45.081000  47.011000      52.03   
Brazil            NaN     49.5650        NaN  44.499706  44.781111        NaN   

                                                   
rank_level Second Tier Third Tier Top Tier    All  
country                                            
Argentina          NaN        NaN    45.66  45.66  
Australia        50.40      47.47    45.97  51.61  
Austria            NaN      47.78    46.29  47.78  
Belgium          49.73      47.14    46.21  52.03  
Brazil           49.82        NaN    46.08  49.82

A pivot table is just a multi-level DataFrame, and we can acess Series or cells in the DataFrame in a similar way as we do so for a regular DataFrame.

In [29]:
new_df = wur.pivot_table(values='score', index='country', columns='rank_level', aggfunc=[np.mean, np.max], margins=True)

# Function to return the top value
new_df['mean']['First Tier'].idxmax()

'United Kingdom'

## Stacking

Moving the lowermost column to the innermost row.

In [30]:
stacked = new_df.stack()
stacked.head()

mean   amax
country   rank_level                   
Argentina Top Tier     44.672857  45.66
          All          44.672857  45.66
Australia First Tier   47.942500  51.61
          Second Tier  49.242500  50.40
          Third Tier   47.285000  47.47

In the original pivot table, rank levels are the lowermost column, after stacking, rank levels become the innermost index, appearing to the right after country.

In [31]:
# Unstacking
unstacked = stacked.unstack()
unstacked.head()

mean                                                    amax  \
rank_level First Tier Second Tier Third Tier   Top Tier        All First Tier   
country                                                                         
All         58.350675    49.06545  46.843450  44.738871  47.798395     100.00   
Argentina         NaN         NaN        NaN  44.672857  44.672857        NaN   
Australia   47.942500    49.24250  47.285000  44.645750  45.825517      51.61   
Austria           NaN         NaN  47.066667  44.864286  45.139583        NaN   
Belgium     51.875000    49.08400  46.746667  45.081000  47.011000      52.03   

                                                    
rank_level Second Tier Third Tier Top Tier     All  
country                                             
All              51.29      47.93    46.34  100.00  
Argentina          NaN        NaN    45.66   45.66  
Australia        50.40      47.47    45.97   51.61  
Austria            NaN      47.78    46.29   47.78  
Belgium          49.73      47.14    46.21   52.03